In [1]:
from dataclasses import dataclass
import pandas as pd
import random
%matplotlib inline
from sensitivity import SensitivityAnalyzer

In [2]:
#Internal Randomness
@dataclass
class ModelInputs:
    starting_salary: tuple = (40000,60000,70000)
    promos_every_n_years: tuple = (8,5,4)
    cost_of_living_raised: tuple = (0.01,0.02,0.03)
    promo_raise: tuple =(0.07,0.15,0.2)
    savings_rate: tuple =(0.15,0.25,0.35)
    interest_rate: tuple =(0.03,0.05,0.06)
    desired_cash: int = 1500000
    recession_prob: float =0.2
    expansion_prob: float =0.3
    case_names: tuple=('Recession','Normal','Expansion')
    n_iter: int=1000
model_data = ModelInputs()
model_data

ModelInputs(starting_salary=(40000, 60000, 70000), promos_every_n_years=(8, 5, 4), cost_of_living_raised=(0.01, 0.02, 0.03), promo_raise=(0.07, 0.15, 0.2), savings_rate=(0.15, 0.25, 0.35), interest_rate=(0.03, 0.05, 0.06), desired_cash=1500000, recession_prob=0.2, expansion_prob=0.3, case_names=('Recession', 'Normal', 'Expansion'), n_iter=1000)

In [3]:
def get_economy_case_number(data):
    normal_probability=1-model_data.recession_prob-model_data.expansion_prob
    case_num=random.choices([0,1,2],weights=[model_data.recession_prob,normal_probability,model_data.expansion_prob])[0]
    return case_num
get_economy_case_number(model_data)

1

In [4]:
case=get_economy_case_number(model_data)
case

2

In [5]:
for i in range(6):
    year=i+1
    num_promos = int(year / model_data.promos_every_n_years[case])
    salary_t = model_data.starting_salary[case] * (1 + model_data.cost_of_living_raised[case]) ** year * (1 + model_data.promo_raise[case]) ** num_promos
    print(f' The Salary at year {year} is ${salary_t:,.0f}.')

 The Salary at year 1 is $72,100.
 The Salary at year 2 is $74,263.
 The Salary at year 3 is $76,491.
 The Salary at year 4 is $94,543.
 The Salary at year 5 is $97,379.
 The Salary at year 6 is $100,300.


In [6]:
def salary_at_year(modelInputs,year,case):
    num_promos = int(year / modelInputs.promos_every_n_years[case])
    salary_t = modelInputs.starting_salary[case] * (1 + modelInputs.cost_of_living_raised[case]) ** year * (1 + modelInputs.promo_raise[case]) ** num_promos
    return salary_t

In [7]:
salary_at_year(model_data,20,2)

314592.7895216818

In [8]:
def cash_saved_during_year(model_data,year,case):
        salary= salary_at_year(model_data,year,case)
        cash_saved=salary*model_data.savings_rate[case]
        return cash_saved

In [9]:
def wealth_at_year(model_data,year,prior_wealth,case):
        cash_saved = cash_saved_during_year(model_data,year,case)
        wealth = prior_wealth*(1 + model_data.interest_rate[case]) + cash_saved
        return wealth

In [10]:
wealth_at_year(model_data,20,40000,2)

152507.47633258862

In [11]:
def years_to_retirement(model_data: ModelInputs,print_output=False):
    
    # starting with no cash saved
    prior_wealth = 0  
    wealth = 0
    
    year = 0  # will become 1 on first loop
    
    def salary_at_year(modelInputs,year,case):
        num_promos = int(year / modelInputs.promos_every_n_years[case])
        salary_t = modelInputs.starting_salary[case] * (1 + modelInputs.cost_of_living_raised[case]) ** year * (1 + modelInputs.promo_raise[case]) ** num_promos
        return salary_t
    
    def cash_saved_during_year(model_data,year,case):
        salary= salary_at_year(model_data,year,case)
        cash_saved=salary*model_data.savings_rate[case]
        return cash_saved
    
    def wealth_at_year(model_data,year,prior_wealth,case):
        cash_saved = cash_saved_during_year(model_data,year,case)
        wealth = prior_wealth*(1 + model_data.interest_rate[case]) + cash_saved
        return wealth


    if print_output:
        print('Wealths over time:') # \n makes a blank line in the output.
    while wealth < model_data.desired_cash:
        year = year + 1
        case=get_economy_case_number(model_data)
        case_type=model_data.case_names[case]
        wealth = wealth_at_year(model_data, year, prior_wealth,case)
        if print_output:
            print(f'The wealth at year {year} ({case_type} economy) is ${wealth:,.0f}.')
        # Set next year's prior wealth to this year's wealth
        prior_wealth = wealth
        
    # Now we have exited the while loop, so wealth must be >= desired_cash. Whatever last year was set
    # is the years to retirement.
    if print_output:
        print(f'\nRetirement:\nIt will take {year} years to retire.')  # \n makes a blank line in the output.
    return year
years = years_to_retirement(model_data,print_output=True)

Wealths over time:
The wealth at year 1 (Recession economy) is $6,060.
The wealth at year 2 (Recession economy) is $12,362.
The wealth at year 3 (Expansion economy) is $39,876.
The wealth at year 4 (Normal economy) is $58,106.
The wealth at year 5 (Expansion economy) is $95,675.
The wealth at year 6 (Normal economy) is $119,885.
The wealth at year 7 (Recession economy) is $129,915.
The wealth at year 8 (Normal economy) is $156,622.
The wealth at year 9 (Normal economy) is $185,068.
The wealth at year 10 (Expansion economy) is $243,585.
The wealth at year 11 (Recession economy) is $258,056.
The wealth at year 12 (Recession economy) is $273,031.
The wealth at year 13 (Expansion economy) is $351,585.
The wealth at year 14 (Expansion economy) is $436,717.
The wealth at year 15 (Recession economy) is $457,272.
The wealth at year 16 (Expansion economy) is $566,233.
The wealth at year 17 (Expansion economy) is $684,177.
The wealth at year 18 (Recession economy) is $712,919.
The wealth at year

In [12]:
data=ModelInputs(n_iter=1000)
all_ytrs=[]
for i in range(data.n_iter):
    ytr=years_to_retirement(data,print_output=False)
    all_ytrs.append(ytr)
df=pd.DataFrame()
df['Years to Retirement']=all_ytrs

In [13]:
df

,Years to Retirement
0,28
1,29
2,25
3,24
4,25
...,...
995,24
996,27
997,25
998,27


In [14]:
def sumarize_ytr_df(ModelInputs,desired_cash):
    data=ModelInputs(n_iter=1000,desired_cash=desired_cash)
    for i in range(data.n_iter):
        ytr=years_to_retirement(data,print_output=False)
        all_ytrs.append(ytr)
        
    df=pd.DataFrame()
    df['Years to Retirement']=all_ytrs
    avg_ytr=df['Years to Retirement'].mean()
    std_ytr=df['Years to Retirement'].std()
    max_ytr=df['Years to Retirement'].max()
    min_ytr=df['Years to Retirement'].min()
    print(f'It will take {avg_ytr:.0f} years to retire on average, with standard deviation'
          f' of {std_ytr:.1f},max of {max_ytr:.0f}, and min of {min_ytr:.0f}'
         )


In [15]:
sumarize_ytr_df(ModelInputs,desired_cash=25000000)

It will take 43 years to retire on average, with standard deviation of 17.1,max of 67, and min of 22
